# 05 - Dictionaries (Teams & Players)
Génération et aperçu des dictionnaires équipes/joueurs.


In [1]:
import os
import sys
from pathlib import Path
ROOT = Path(os.getcwd()).resolve()
if (ROOT / 'src').exists():
    sys.path.append(str(ROOT))
elif (ROOT.parent / 'src').exists():
    sys.path.append(str(ROOT.parent))
else:
    sys.path.append(str(ROOT))

raw_candidates = [
    ROOT / 'data/raw/data.json',
    ROOT.parent / 'data/raw/data.json',
]
raw_path = next((p for p in raw_candidates if p.exists()), None)
if raw_path is None:
    raise FileNotFoundError(f'data/raw/data.json introuvable. Cherché: {raw_candidates}')

processed_dir = ROOT / 'data/processed'
processed_dir.mkdir(parents=True, exist_ok=True)

import polars as pl
from src.dota_data import read_processed_tables, write_parquet_tables, build_team_dictionary, build_player_dictionary

if not (processed_dir / 'matches.parquet').exists():
    write_parquet_tables(raw_path, processed_dir)

tables = read_processed_tables(processed_dir)
teams = build_team_dictionary(tables['matches'])
players = build_player_dictionary(tables['players'])

out_dir = ROOT / 'data/dictionaries'
out_dir.mkdir(parents=True, exist_ok=True)
teams_path = out_dir / 'teams.parquet'
players_path = out_dir / 'players.parquet'
teams.write_parquet(teams_path)
players.write_parquet(players_path)

teams.shape, players.shape


((17, 5), (81, 4))

In [2]:
teams.head(10)


team_id,name,tag,logo_url,side_sampled
i64,str,str,str,str
8291895,"""Tundra Esports""","""Tundra""","""https://cdn.steamusercontent.c…","""dire"""
9927796,"""Tearlaments""","""TT""","""https://cdn.steamusercontent.c…","""radiant"""
9351740,"""Yakult Brothers""","""YB""","""https://cdn.steamusercontent.c…","""radiant"""
9467224,"""Aurora Gaming""","""Aurora""","""https://cdn.steamusercontent.c…","""radiant"""
9247354,"""Team Falcons""","""FLCN""","""https://cdn.steamusercontent.c…","""dire"""
9823272,"""Team Yandex""","""TY""","""https://cdn.steamusercontent.c…","""radiant"""
7119388,"""Team Spirit""","""TSpirit""","""https://cdn.steamusercontent.c…","""dire"""
8261500,"""Xtreme Gaming""","""XG""","""https://cdn.steamusercontent.c…","""radiant"""
9640842,"""Team Tidebound""","""TT""","""https://cdn.steamusercontent.c…","""radiant"""


In [3]:
players.head(10)


account_id,matches_played,personaname,name
i64,u32,str,str
898455820,50,"""Буська""","""Malr1ne"""
100058342,50,"""Falcons Malr1ne prime""","""skiter"""
183719386,50,"""SumaoKy-""","""AMMAR_THE_F"""
25907144,50,"""ik lavet penge""","""Cr1t-"""
10366616,50,"""Sneyking""","""Sneyking"""
136829091,10,"""NongMon""","""Whitemon"""
331855530,10,"""poppin""","""Pure"""
346412363,10,"""Ari""","""Ari"""
86698277,10,"""not human (животное)""","""33"""
